Unifying data from all countries from the World Bank Open Data

In [5]:
import pandas as pd

# Define file paths (excluding unemployment_cost_index)
filepaths = {
    'unemployment': '../World_Datasets/Unemployment.csv',
    'current_account_balance': '../World_Datasets/Current_Account_Balance.csv',
    'exchange_rate_usd': '../World_Datasets/Exchange_Rate_USD.csv',
    'gdp': '../World_Datasets/GDP.csv',
    'government_debt': '../World_Datasets/Government_Debt.csv',
    'inflation': '../World_Datasets/Inflation.csv',
    'total_reserves': '../World_Datasets/Total_Reserves.csv'}

# Load all datasets into a dictionary of DataFrames
dataframes = {}
for name, filepath in filepaths.items():
    df = pd.read_csv(filepath)
    
    # Remove 'Series Name' and 'Series Code' columns if they exist
    df = df.drop(columns=['Series Name', 'Series Code'], errors='ignore')
    
    # Rename columns to include the dataset name and year
    df.columns = [
        col if col == 'Country Name' else f"{name}_{col.split(' ')[0]}" if '[' in col else f"{name}_{col}" 
        for col in df.columns]
    # Store the processed DataFrame
    dataframes[name] = df

# Identify the dataset with the least rows
least_rows_key = min(dataframes, key=lambda k: len(dataframes[k]))
print(f"Dataset with the least rows: {least_rows_key}")

# Use the dataset with the least rows as the base
merged_df = dataframes[least_rows_key]

# Perform inner joins with all other datasets on the 'Country Name' column
for name, df in dataframes.items():
    if name != least_rows_key:
        merged_df = pd.merge(merged_df, df, on=f'Country Name', how='inner')

# Keep only one 'Country Code' column
country_code_columns = [col for col in merged_df.columns if 'Country Code' in col]
if country_code_columns:
    merged_df = merged_df.drop(columns=country_code_columns[1:], errors='ignore')  # Drop duplicates, keep the first

# Save the merged dataset to a new CSV file
merged_df.to_csv('../World_Datasets/world_data.csv', index=False)

print("Merged dataset saved successfully as 'world_data.csv'")


Dataset with the least rows: unemployment
Merged dataset saved successfully as 'world_data.csv'


Cleaning the S&P Index Data (our target Y)

In [18]:
indexes = pd.read_csv('../World_Datasets/S&P_Index.csv')
indexes = indexes[indexes['Country'] != 'European Union'] #dropping as is not a country
indexes = indexes.dropna() #dropping null values (for which we don't have a rating)
indexes = indexes.reset_index(drop=True) #putting the index back into place
indexes

,Country,S&P Rating
0,Australia,AAA
1,Canada,AAA
2,Denmark,AAA
3,Germany,AAA
4,Liechtenstein,AAA
...,...,...
130,Sri Lanka,SD
131,Ukraine,SD
132,Lebanon,D
133,Puerto Rico,D


Using the data cleaned, with missing data imputed by chatgpt. 

In [13]:
data = pd.read_csv('../World_Datasets/world_data_cleaned_imputed.csv')

Replacing the country names to have the same in both datasets. We will be using the names from the ratings.

In [14]:
#replacing the country names 
data = data.replace('Korea Rep.', 'South Korea')
data = data.replace('Cabo Verde', 'Cape Verde')
data = data.replace('Congo Dem. Rep.', 'Congo')
data = data.replace('Czechia', 'Czech Republic')
data = data.replace('Egypt Arab Rep.', 'Egypt')
data = data.replace("Cote d'Ivoire", 'Ivory Coast')
data = data.replace('Kyrgyz Republic', 'Kyrgyzstan')
data = data.replace('Congo Rep.', 'Republic of the Congo')
data = data.replace('Russian Federation', 'Russia')
data = data.replace('Slovak Republic', 'Slovakia')
data = data.replace('Turkiye', 'Turkey')
data = data.replace('Viet Nam', 'Vietnam')

In [19]:
data

,Country Name,country_code,unemployment_2000,unemployment_2014,unemployment_2015,unemployment_2016,unemployment_2017,unemployment_2018,unemployment_2019,unemployment_2020,...,total_reserves_2016,total_reserves_2017,total_reserves_2018,total_reserves_2019,total_reserves_2020,total_reserves_2021,total_reserves_2022,total_reserves_2023,Unnamed: 73,value_counts
0,Afghanistan,AFG,7.955000,7.915000,9.011000,10.100000,11.184000,11.206000,11.224000,11.710000,...,7.281911e+09,8.097281e+09,8.206682e+09,8.497656e+09,9.748946e+09,7.799157e+09,5.849368e+09,3.899579e+09,1.949789e+09,54.000000
1,Albania,ALB,19.023000,18.055000,17.193000,15.418000,13.616000,12.304000,11.466000,12.790000,...,3.108607e+09,3.588584e+09,3.893560e+09,3.763661e+09,4.817386e+09,5.634561e+09,5.266464e+09,6.455446e+09,3.227723e+09,71.000000
2,Algeria,DZA,29.770000,10.207000,11.206000,10.202000,12.000000,12.150000,12.285000,14.040000,...,1.207881e+11,1.048525e+11,8.738291e+10,7.179595e+10,5.943439e+10,5.621101e+10,7.185229e+10,8.121674e+10,4.060837e+10,73.000000
3,American Samoa,ASM,9.640537,9.136085,9.052059,8.903803,8.664479,8.383856,8.244777,9.305229,...,2.903333e+08,2.580741e+08,2.258148e+08,1.935556e+08,1.612963e+08,1.290371e+08,9.677780e+07,6.451854e+07,3.225928e+07,20.000000
4,Andorra,AND,9.640537,9.136085,9.052059,8.903803,8.664479,8.383856,8.244777,9.305229,...,1.310216e+09,1.164636e+09,1.019057e+09,8.734771e+08,7.278976e+08,5.823181e+08,4.367386e+08,2.911590e+08,1.455795e+08,21.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,Yemen,YEM,11.561000,13.467000,17.900000,18.382000,18.564000,17.644000,17.303000,17.972000,...,8.774422e+08,9.205589e+08,2.983208e+09,1.653643e+09,9.696130e+08,1.688000e+09,1.250826e+09,8.338843e+08,4.169422e+08,48.000000
214,Zambia,ZMB,12.930000,8.131000,8.316000,8.418000,8.518000,5.031000,5.539000,6.032000,...,2.352719e+09,2.082084e+09,1.569230e+09,1.448624e+09,1.203448e+09,2.753875e+09,2.967565e+09,1.978377e+09,9.891885e+08,72.000000
215,Zimbabwe,ZWE,5.682000,4.774000,5.383000,5.893000,6.347000,6.792000,7.373000,8.621000,...,4.071932e+08,2.926212e+08,8.695109e+07,1.512405e+08,3.340502e+07,8.387802e+08,5.986216e+08,1.155297e+08,5.776489e+07,68.000000
216,NaN,NaN,9.640537,9.136085,9.052059,8.903803,8.664479,8.383856,8.244777,9.305229,...,5.399610e+10,5.797643e+10,5.782644e+10,6.070521e+10,6.637602e+10,6.978272e+10,6.490856e+10,6.784714e+10,3.392357e+10,63.705069


Merging the datasets 

In [20]:
merged_df = pd.merge(indexes, data, left_on='Country', right_on='Country Name', how='inner')

merged_df.to_csv('../World_Datasets/final_dataset_world.csv', index=False)

print("Merged dataset saved successfully as 'final_dataset_world.csv'")

Merged dataset saved successfully as 'final_dataset_world.csv'
